In [ ]:
# -----********************-----

# Created Time: 2025/07/13

# Last updated: 2025/07/13

# Author: Yiyi He

### Use Case

# This notebook explores the application of autoregressive models
# 1. 

# -----********************-----

# Libraries

In [1]:
# Import libraries
import os
import warnings
from pathlib import Path
warnings.filterwarnings("ignore")

# Stats
from statsmodels.tsa.api import ARDL
import statsmodels.api as sm
import numpy as np
from statsmodels.tsa.ardl import ardl_select_order
from statsmodels.tsa.stattools import grangercausalitytests

# Geo
from shapely.geometry import Point, Polygon
# import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.display.max_rows = 1000

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# Processing
from tqdm import tqdm
import functools as ft

# Granger Causality

## GC with hourly data

In [5]:
home_dir = '/Users/yiyi/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology(2)/Research/Energy_resilience/'

hourly_df = pd.read_csv(home_dir + "01_data/processed/csv/hourly_519station_3weather.csv",
                        index_col=0)

In [ ]:
# Create a list of unique station ids
station_id_lst = list(set(hourly_df.station_id.unique()))
# Set max lag
maxlag = 72
# Target
target = "pct_blackout"
# Predictor
predictor = "tp"

# Initiate an empty dictionary
tp_gc_max72_dic = {}

# List of stations visited
# existing_keys = station_id_t2m_gc_max72_dic.keys()
# Iterate through all stations
for s_id in tqdm(station_id_lst):
    if s_id == 518 or s_id == 563:
        continue 
    else:
        # Subset station data
        station_df = hourly_df[hourly_df['station_id'] == s_id].sort_values(by='datetime')
        test_result = grangercausalitytests(
            station_df[[target, predictor]], maxlag=maxlag, addconst=True, verbose=False)
        F_test_p_values = [round(test_result[i+1][0]['ssr_ftest'][1],4) for i in range(maxlag)]
        Chi_squared_p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]
        p_values_min = np.min(F_test_p_values+Chi_squared_p_values)
        # Key: station id, Value: list of 1. minimum F/Chi p values 2. F-test p values for all lags 3. Chi-square test p-values for all lags
        tp_gc_max72_dic[s_id] = [p_values_min, F_test_p_values, Chi_squared_p_values]

  1%|          | 6/519 [00:36<52:55,  6.19s/it]  